### Edge histograms and difference edge images for daytime images and their night-time transformations  ###

In [1]:
dirs_images = [
    "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_augmented_v032_e14/train_A_ganaug_050",
    "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/train_A",
    "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/train_B",
    "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/train_C",
    "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/train_dev_A",
    "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/train_dev_B",
    "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/train_dev_C",
]

out_dir = "/home/SharedFolder/CurrentDatasets/bdd_edgemaps"

suffix_augmented_images = "_transfer_AtoB"

In [ ]:
import os
import glob
image_paths = [glob.glob(dir_images + "/*.jpg") for dir_images in dirs_images]
image_paths = [item for sublist in image_paths for item in sublist]
image_paths_augmented = [image_path for image_path in image_paths if suffix_augmented_images in image_path]
image_uuids_augmented = [image_path.split(os.sep)[-1].split(suffix_augmented_images)[0] for image_path in image_paths_augmented]
image_paths_not_augmented = [image_path for image_path in image_paths if suffix_augmented_images not in image_path]
image_uuids_not_augmented = [image_path.split(os.sep)[-1].split(".jpg")[0] for image_path in image_paths_not_augmented]

dict_aug2naug = {}
for i in range(len(image_uuids_augmented)):
    if image_uuids_augmented[i] not in dict_aug2naug:
        if image_uuids_augmented[i] in image_uuids_not_augmented:
            idx = image_uuids_not_augmented.index(image_uuids_augmented[i])
            dict_aug2naug[image_paths_augmented[i]] = image_paths_not_augmented[idx]

In [ ]:
import cv2
import pandas as pd
import numpy as np
import shutil

if os.path.isdir(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir, exist_ok =  True)
    
df_histograms_aug = pd.DataFrame(columns = list(range(-2040, 2040)))
df_histograms_naug = pd.DataFrame(columns = list(range(-2040, 2040)))

cntr = 0
for img_aug_path, img_naug_path in dict_aug2naug.items():
    img_aug = cv2.imread(img_aug_path)
    img_aug = cv2.cvtColor(img_aug, cv2.COLOR_BGR2GRAY)
    img_naug = cv2.imread(img_naug_path)
    img_naug = cv2.cvtColor(img_naug, cv2.COLOR_BGR2GRAY)   

    laplacian64f_aug = cv2.Laplacian(img_aug,cv2.CV_64F, ksize = 3)
    laplacian64f_naug = cv2.Laplacian(img_naug,cv2.CV_64F, ksize = 3)  
   
    norm_abs_diff_laplacian_64f_aug_naug = cv2.normalize(np.abs(laplacian64f_aug - laplacian64f_naug), None, 0, 255, cv2.NORM_MINMAX)
    norm_abs_diff_laplacian_8u_aug_naug = np.uint8(norm_abs_diff_laplacian_64f_aug_naug)
    
    cv2.imwrite(os.path.join(out_dir, img_naug_path.split(os.sep)[-1].split(".jpg")[0] + "_AUGminusNAUG.jpg"), norm_abs_diff_laplacian_8u_aug_naug, [int(cv2.IMWRITE_JPEG_QUALITY), 95]) 
  
    hist_aug, bins_aug = np.histogram(laplacian64f_aug.ravel(), 4080, [-2040, 2040])
    hist_naug, bins_naug = np.histogram(laplacian64f_naug.ravel(), 4080, [-2040, 2040])  
        
    df_histograms_aug.loc[cntr] = hist_aug
    df_histograms_naug.loc[cntr] = hist_naug
    
    cntr += 1
    
    if cntr % 100 == 0:
        print(f"Processed image {cntr} of {len(dict_aug2naug)}.")
        
df_histograms_aug.to_csv(os.path.join(out_dir, "aug_histograms.csv"), header = True)
df_histograms_naug.to_csv(os.path.join(out_dir, "naug_histograms.csv"), header = True)

Processed image 100 of 15353.
Processed image 200 of 15353.
Processed image 300 of 15353.
Processed image 400 of 15353.
Processed image 500 of 15353.
Processed image 600 of 15353.
Processed image 700 of 15353.
Processed image 800 of 15353.
Processed image 900 of 15353.
Processed image 1000 of 15353.
Processed image 1100 of 15353.
Processed image 1200 of 15353.
Processed image 1300 of 15353.
Processed image 1400 of 15353.
Processed image 1500 of 15353.
Processed image 1600 of 15353.
Processed image 1700 of 15353.
Processed image 1800 of 15353.
Processed image 1900 of 15353.
Processed image 2000 of 15353.
Processed image 2100 of 15353.
Processed image 2200 of 15353.
Processed image 2300 of 15353.
Processed image 2400 of 15353.
Processed image 2500 of 15353.
Processed image 2600 of 15353.
Processed image 2700 of 15353.
Processed image 2800 of 15353.
Processed image 2900 of 15353.
Processed image 3000 of 15353.
Processed image 3100 of 15353.
Processed image 3200 of 15353.
Processed image 3

### Histograms for real night images ###

In [ ]:
real_night_time_json = "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/annotations/bdd100k_sorted_train_C_over.json"
real_night_time_images = "/home/SharedFolder/CurrentDatasets/bdd100k_sorted_coco/train_C"

In [ ]:
df_bdd = pd.read_json(real_night_time_json)
df_bdd = df_bdd.reset_index(drop = True)
df_bdd["timeofday"] = df_bdd["attributes"].apply(lambda x: x["timeofday"])
df_bdd = df_bdd.loc[df_bdd.timeofday == "night"]
df_bdd = df_bdd.drop(columns=["timeofday", "attributes", "labels", "timestamp"]).reset_index(drop = True)
nighttime_files = df_bdd["name"].tolist()

In [ ]:
df_histograms_night = pd.DataFrame(columns = list(range(-2040, 2040)))

cntr = 0

for nighttime_file in nighttime_files:
    img_night = cv2.imread(real_night_time_images + "/" + nighttime_file)
    img_night = cv2.cvtColor(img_night, cv2.COLOR_BGR2GRAY)

    laplacian64f_night = cv2.Laplacian(img_night,cv2.CV_64F, ksize = 3)

    hist_night, bins_night = np.histogram(laplacian64f_night.ravel(), 4080, [-2040, 2040])
        
    df_histograms_night.loc[cntr] = hist_night
    
    cntr += 1
    
    if cntr % 100 == 0:
        print(f"Processed image {cntr} of {len(nighttime_files)}.")
        
df_histograms_night.to_csv(os.path.join(out_dir, "night_histograms.csv"), header = True)